In [ ]:
# Tutorial 6: Population Level Modeling (with PopNet)

In [ ]:
from bmtk.builder import NetworkBuilder

net = NetworkBuilder('V1')

# 构造两个总体
net.add_nodes(
    pop_name='excitatory',                # 总体的名称（name of the population）
    ei='e',                               # 可选
    location='VisL4',                     # 可选
    model_type='population',              # 必须，表示模型的类型
    model_template='dipde:Internal',      # 必须，指导创建哪些DiPDE对象
    dynamics_params='exc_model.json'      # 必须，参数文件
)

net.add_nodes(
    pop_name='inhibitory',
    ei='i',
    model_type='population',
    model_template='dipde:Internal',
    dynamics_params='inh_model.json'
)

In [ ]:
# 构造连接
net.add_edges(
    source={'ei': 'e'}, target={'ei': 'i'},
    syn_weight=0.005,
    nsyns=20,
    delay=0.002,
    dynamics_params='ExcToInh.json'
)

net.add_edges(
    source={'ei': 'i'}, target={'ei': 'e'},
    syn_weight=-0.002,
    nsyns=10,
    delay=0.002,
    dynamics_params='InhToExc.json'
)

In [ ]:
net.build()
net.save(output_dir='sim_ch06/network')

In [ ]:
# 创建外部节点（单节点网络）
input_net = NetworkBuilder('LGN')
input_net.add_nodes(
    pop_name='tON',
    ei='e',
    model_type='virtual'
)

input_net.add_edges(
    target=net.nodes(ei='e'),
    syn_weight=0.0025,
    nsyns=10,
    delay=0.002,
    dynamics_params='input_ExcToExc.json'
)

input_net.build()
input_net.save(output_dir='sim_ch06/network')

In [ ]:
! python -m bmtk.utils.sim_setup -n network --tstop 1500.0 popnet sim_ch06/
! cd sim_ch06
! wget https://github.com/AllenInstitute/bmtk/raw/develop/examples/pop_2pops/inputs/external_rates.csv -O lgn_rates.csv

In [ ]:
# 创建环境+运行模拟
from bmtk.simulator import popnet

configure = popnet.config.from_json('sim_ch06\config.json')
configure.build_env()
network = popnet.PopNetwork.from_config(configure)
sim = popnet.PopSimulator.from_config(configure, network)
sim.run()

In [ ]:
# 分析
from bmtk.analyzer.firing_rates import plot_rates_popnet

plot_rates_popnet('sim_ch06/network/V1_node_types.csv', 'sim_ch06/output/firing_rates.csv', model_keys='pop_name')